In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
timer.start()
classificacao_final = ['0','1','2','3','5']
# classificacao_final = ['0','1']
# classificacao_final = ['2']

notifica = Notifica()
notifica.read(download_metabase(filename='null.csv',where=f"classificacao_final IS NULL"))
# notifica.read(join('input','queries','null.csv'))
for cf in classificacao_final:
    notifica.read(download_metabase(filename=f"{cf}.csv",where=f"classificacao_final = {cf}"), append=True)
    # notifica.read(join('input','queries',f"{cf}.csv"), append=True)

notifica.save()
# notifica.load()
print(timer.ftime())
notifica.shape()

Requesting notifica where classificacao_final IS NULL
Success code 202
Saving in input\queries\null.csv
Download finish, time elapsed: 0:00:33.316461

Requesting notifica where classificacao_final = 0
Success code 202
Saving in input\queries\0.csv
Download finish, time elapsed: 0:00:17.730910

Requesting notifica where classificacao_final = 1
Success code 202
Saving in input\queries\1.csv
Download finish, time elapsed: 0:00:30.045171

Requesting notifica where classificacao_final = 2
Success code 202
Saving in input\queries\2.csv
Download finish, time elapsed: 0:00:39.701233

Requesting notifica where classificacao_final = 3
Success code 202
Saving in input\queries\3.csv
Download finish, time elapsed: 0:00:50.468389

Requesting notifica where classificacao_final = 5
Success code 202
Saving in input\queries\5.csv
Download finish, time elapsed: 0:00:40.254290

00:22:42.11


(2236850, 1028142, 10524, 1196095, 2089)

In [3]:
timer.start()
casos_confirmados = CasosConfirmados()
casos_confirmados.read()
casos_confirmados.save()
# casos_confirmados.load()
print(timer.ftime())
casos_confirmados.shape()

00:02:16.01


(568350, 0, 0, 0)

In [4]:
casosn = notifica.get_casos()
casosc = casos_confirmados.get_casos()
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,541686
CASO DESCARTADO,762669
CASO SUSPEITO,924840
IGNORADO,643
SINDROME GRIPAL NAO ESPECIFICADA,7012


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 499891
201394 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14316
CASO DESCARTADO,122749
CASO SUSPEITO,159991
IGNORADO,48
SINDROME GRIPAL NAO ESPECIFICADA,1393


In [6]:
# casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 538600
197876 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,20370
CASO DESCARTADO,142411
CASO SUSPEITO,176355
IGNORADO,38
SINDROME GRIPAL NAO ESPECIFICADA,1550


In [8]:
# casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo data_nascimento: 575340
233693 pacientes pelo data_nascimento que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,17323
CASO DESCARTADO,138677
CASO SUSPEITO,184015
IGNORADO,49
SINDROME GRIPAL NAO ESPECIFICADA,1583


In [10]:
# casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_resid: 469361
191899 pacientes pelo hash_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14888
CASO DESCARTADO,112381
CASO SUSPEITO,148959
IGNORADO,35
SINDROME GRIPAL NAO ESPECIFICADA,1199


In [12]:
# casosn = casosn.drop(index=casos_duplicados)

In [13]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_atend: 402980
163855 pacientes pelo hash_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,11614
CASO DESCARTADO,99733
CASO SUSPEITO,126626
IGNORADO,18
SINDROME GRIPAL NAO ESPECIFICADA,1134


In [14]:
# casosn = casosn.drop(index=casos_duplicados)

In [15]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'casos_duplicados_removidos.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'casos_duplicados_mantidos.csv'), index=False)

Das 2236850 notificações baixadas, 0 (385649) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 2236850


In [16]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)
casos_confirmados_nao_notifica.shape

(27978, 23)

In [17]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)
casos_confirmados_notifica.shape

(627878, 53)

In [24]:
casos_confirmados_notifica.loc[casos_confirmados_notifica['hash_diag']=='31082020']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag
355104,814450,75170970000134-113696,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355115,814472,75170970000134-113698,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355147,814574,75170970000134-113690,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355149,814576,75170970000134-113694,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355151,814580,75170970000134-113693,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355152,814581,75170970000134-113686,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355153,814584,75170970000134-113685,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,1,CASO SUSPEITO,3,EM INVESTIGACAO,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,13,NAO REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
1070671,814585,75170970000134-113683,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,12,REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
1070672,814588,75170970000134-113680,1900-01-01,None,None,4,CPF NAO INFORMADO,120,M,99,IGNORADO,0,,35,355030,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,11,COVID-19 IGG/IGM,12,REAGENTE,NaT,NaT,NaT,1,ABERTA,2,12,LAB HILAB,41,410690,2020-08-31,NaT,2020-08-31,SAO PAULO,SP,None,CURITIBA,PR,120SAOPAULO,120CURITIBA,NaN,01011900,31082020
355156,814589,75170970000134-113674

In [18]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]

In [19]:
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2]
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_status_notificacao'].isin([1,2])]

notifica_nao_casos_confirmados['month'] = notifica_nao_casos_confirmados.apply(lambda x: x['data_notificacao'].month, axis=1)
nnccm = notifica_nao_casos_confirmados.groupby(by='month')[['id']].count()
nnccm.index = [ meses[int(x)-1] for x in nnccm.index] 
nnccm

,id
jan,572
fev,757
mar,41
abril,106
mai,142
jun,454
jul,530
ago,377
set,202
out,149


In [20]:
notifica_nao_casos_confirmados.to_excel(join(output,'notifica_nao_casos_confirmados.xlsx'), index=False)
notifica_nao_casos_confirmados.shape

(3982, 54)